In [4]:
from cryocat import cryomap
from pathlib import Path
from PIL import Image
import imageio

mrc = cryomap.read('tutorial_data/experimental_TS/0248_dose-filt_ali_bin8.mrc')
mrc = ((mrc - mrc.min()) * (1/(mrc.max() - mrc.min()) * 255)).astype('uint8')

PATH_ORIG = f'tutorial_data/orig'
Path(PATH_ORIG).mkdir(parents=True, exist_ok=True)

for i in range(0, mrc.shape[2]):
    image = Image.fromarray(mrc[:,:,i])
    if image.mode != 'RGB':
        image = image.convert('RGB')
    imageio.imwrite(f'{PATH_ORIG}/{(i*10):04}.png', image)

In [5]:
import subprocess as sp
import shutil
import os

PATH_ORIG = 'tutorial_data/orig'

PATH_BASE = 'tutorial_data/tilts_cryoTIGER_model'

process = sp.Popen(f'python3 -m eval.interpolator_cli --pattern "{PATH_ORIG}" \
                    --model_path models/cryoTIGER_model \
                    --times_to_interpolate 1', shell=True)
process.wait()

if os.path.exists("tutorial_data/interpolated_frames_cryoTIGER_model"):
    shutil.rmtree("tutorial_data/interpolated_frames_cryoTIGER_model")
shutil.move(f"{PATH_ORIG}/interpolated_frames", "tutorial_data/interpolated_frames_cryoTIGER_model")

process = sp.Popen(f'python3 -m eval.interpolator_cli --pattern "{PATH_ORIG}" \
                    --model_path models/pre-trained_Vimeo-90K_data_model \
                    --times_to_interpolate 1', shell=True)
process.wait()

if os.path.exists("tutorial_data/interpolated_frames_pre-trained_model"):
    shutil.rmtree("tutorial_data/interpolated_frames_pre-trained_model")
shutil.move(f"{PATH_ORIG}/interpolated_frames", "tutorial_data/interpolated_frames_pre-trained_model")


2024-12-13 10:38:44.577568: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-13 10:39:07.547771: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ms/tomajtne/.conda/envs/dl/lib/
2024-12-13 10:39:07.547927: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/ms/tomajtne/.conda/envs/dl/lib/
2024-12-13 10:39:07.547939: W tensorflow/compiler/tf2t

'tutorial_data/interpolated_frames_pre-trained_model'

In [6]:
from cryocat import cryomap
from PIL import Image
from os import listdir
import os
import imageio.v3 as iio
import numpy as np

PATH_GEN = f'tutorial_data/interpolated_frames_cryoTIGER_model/'

_, _, files = next(os.walk(f'{PATH_GEN}'))
    
tomo3d = []
for tomo in sorted(listdir(f'{PATH_GEN}/')):
    img = Image.open(f'{PATH_GEN}/{tomo}').convert('L')
    img.save(f'{PATH_GEN}/{tomo}')
    
    im = iio.imread(f'{PATH_GEN}/{tomo}')
    tomo3d.append(im)
tomo3d = np.stack(tomo3d, axis=2)
cryomap.write(tomo3d, f"tutorial_data/interpolated_TS/demo_tilt_series_bin8_interpolated_cryoTIGER_model.mrc", data_type=np.single)


PATH_GEN = f'tutorial_data/interpolated_frames_pre-trained_model/'

_, _, files = next(os.walk(f'{PATH_GEN}'))
    
tomo3d = []
for tomo in sorted(listdir(f'{PATH_GEN}/')):
    img = Image.open(f'{PATH_GEN}/{tomo}').convert('L')
    img.save(f'{PATH_GEN}/{tomo}')
    
    im = iio.imread(f'{PATH_GEN}/{tomo}')
    tomo3d.append(im)
tomo3d = np.stack(tomo3d, axis=2)
cryomap.write(tomo3d, f"tutorial_data/interpolated_TS/demo_tilt_series_bin8_interpolated_pre-trained_model.mrc", data_type=np.single)
